# Xatu Public Contributors Analysis

This notebook fetches and analyzes public contributor data from Clickhouse and generates JSON files for the frontend.

In [11]:
import os
import json
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine
from pathlib import Path

# Environment variables
CLICKHOUSE_HOST = os.getenv('CLICKHOUSE_HOST')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD')
DATA_DIR = '../data/xatu-public-contributors'

# if not all([CLICKHOUSE_HOST, CLICKHOUSE_USER, CLICKHOUSE_PASSWORD, DATA_DIR]):
#     raise ValueError("Missing required environment variables")

# Create Clickhouse connection
engine = create_engine(
    f'clickhouse+native://{CLICKHOUSE_USER}:{CLICKHOUSE_PASSWORD}@{CLICKHOUSE_HOST}/default'
)

In [12]:
# Create sample data
versions_data = [
    {"client_name": "Lighthouse", "client_version": "v4.5.0", "count": 1250},
    {"client_name": "Prysm", "client_version": "v4.0.7", "count": 980}, 
    {"client_name": "Teku", "client_version": "v23.6.2", "count": 750},
    {"client_name": "Nimbus", "client_version": "v23.6.1", "count": 620},
    {"client_name": "Lighthouse", "client_version": "v4.4.1", "count": 450},
    {"client_name": "Prysm", "client_version": "v4.0.6", "count": 380},
    {"client_name": "Teku", "client_version": "v23.6.1", "count": 320},
    {"client_name": "Nimbus", "client_version": "v23.6.0", "count": 280},
    {"client_name": "Lighthouse", "client_version": "v4.3.0", "count": 220},
    {"client_name": "Prysm", "client_version": "v4.0.5", "count": 180}
]

DATA_DIR
# Save to JSON
with open(Path(DATA_DIR) / 'top_client_versions.json', 'w') as f:
    json.dump(versions_data, f)
